In [ ]:
pip install ta-lib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 19.6 MB/s eta 0:00:00


In [ ]:
pip install yfinance

In [ ]:
import yfinance as yf
import talib
import pandas as pd

# --- 1. The Meaning Dictionary ---
pattern_desc = {
    'CDL2CROWS': 'Bearish Reversal: Two Crows',
    'CDL3BLACKCROWS': 'Bearish Reversal: Three Black Crows',
    'CDL3INSIDE': 'Reversal: Three Inside Up/Down',
    'CDL3LINESTRIKE': 'Continuation: Three-Line Strike',
    'CDL3OUTSIDE': 'Reversal: Three Outside Up/Down',
    'CDL3STARSINSOUTH': 'Bullish Reversal: Three Stars In The South',
    'CDL3WHITESOLDIERS': 'Bullish Reversal: Three White Soldiers',
    'CDLABANDONEDBABY': 'Reversal: Abandoned Baby',
    'CDLADVANCEBLOCK': 'Bearish Reversal: Advance Block',
    'CDLBELTHOLD': 'Reversal: Belt-hold',
    'CDLBREAKAWAY': 'Reversal: Breakaway',
    'CDLCLOSINGMARUBOZU': 'Continuation: Closing Marubozu',
    'CDLCONCEALBABYSWALL': 'Bullish Reversal: Concealing Baby Swallow',
    'CDLCOUNTERATTACK': 'Reversal: Counterattack',
    'CDLDARKCLOUDCOVER': 'Bearish Reversal: Dark Cloud Cover',
    'CDLDOJI': 'Indecision: Doji',
    'CDLDOJISTAR': 'Reversal: Doji Star',
    'CDLDRAGONFLYDOJI': 'Bullish Reversal: Dragonfly Doji',
    'CDLENGULFING': 'Reversal: Engulfing Pattern',
    'CDLEVENINGDOJISTAR': 'Bearish Reversal: Evening Doji Star',
    'CDLEVENINGSTAR': 'Bearish Reversal: Evening Star',
    'CDLGAPSIDESIDEWHITE': 'Continuation: Up/Down-gap side-by-side white lines',
    'CDLGRAVESTONEDOJI': 'Bearish Reversal: Gravestone Doji',
    'CDLHAMMER': 'Bullish Reversal: Hammer',
    'CDLHANGINGMAN': 'Bearish Reversal: Hanging Man',
    'CDLHARAMI': 'Reversal: Harami Pattern',
    'CDLHARAMICROSS': 'Reversal: Harami Cross Pattern',
    'CDLHIGHWAVE': 'Indecision: High-Wave Candle',
    'CDLHIKKAKE': 'Continuation: Hikkake Pattern',
    'CDLHIKKAKEMOD': 'Continuation: Modified Hikkake Pattern',
    'CDLHOMINGPIGEON': 'Bullish Reversal: Homing Pigeon',
    'CDLIDENTICAL3CROWS': 'Bearish Reversal: Identical Three Crows',
    'CDLINNECK': 'Bearish Continuation: In-Neck Pattern',
    'CDLINVERTEDHAMMER': 'Bullish Reversal: Inverted Hammer',
    'CDLKICKING': 'Reversal: Kicking',
    'CDLKICKINGBYLENGTH': 'Reversal: Kicking - bull/bear determined by the longer marubozu',
    'CDLLADDERBOTTOM': 'Bullish Reversal: Ladder Bottom',
    'CDLLONGLEGGEDDOJI': 'Indecision: Long Legged Doji',
    'CDLLONGLINE': 'Continuation: Long Line Candle',
    'CDLMARUBOZU': 'Continuation: Marubozu',
    'CDLMATCHINGLOW': 'Bullish Reversal: Matching Low',
    'CDLMATHOLD': 'Bullish Continuation: Mat Hold',
    'CDLMORNINGDOJISTAR': 'Bullish Reversal: Morning Doji Star',
    'CDLMORNINGSTAR': 'Bullish Reversal: Morning Star',
    'CDLONNECK': 'Bearish Continuation: On-Neck Pattern',
    'CDLPIERCING': 'Bullish Reversal: Piercing Pattern',
    'CDLRICKSHAWMAN': 'Indecision: Rickshaw Man',
    'CDLRISEFALL3METHODS': 'Continuation: Rising/Falling Three Methods',
    'CDLSEPARATINGLINES': 'Continuation: Separating Lines',
    'CDLSHOOTINGSTAR': 'Bearish Reversal: Shooting Star',
    'CDLSHORTLINE': 'Indecision: Short Line Candle',
    'CDLSPINNINGTOP': 'Indecision: Spinning Top',
    'CDLSTALLEDPATTERN': 'Bearish Reversal: Stalled Pattern',
    'CDLSTICKSANDWICH': 'Bullish Reversal: Stick Sandwich',
    'CDLTAKURI': 'Bullish Reversal: Takuri (Dragonfly Doji with very long shadow)',
    'CDLTASUKIGAP': 'Continuation: Tasuki Gap',
    'CDLTHRUSTING': 'Bearish Continuation: Thrusting Pattern',
    'CDLTRISTAR': 'Reversal: Tristar Pattern',
    'CDLUNIQUE3RIVER': 'Bullish Reversal: Unique 3 River',
    'CDLUPSIDEGAP2CROWS': 'Bearish Reversal: Upside Gap Two Crows',
    'CDLXSIDEGAP3METHODS': 'Continuation: Upside/Downside Gap Three Methods'
}

def get_nse_data(symbol, period="3mo"):
    """Fetches data for an NSE stock OR Index."""
    symbol = symbol.upper()
    if symbol.startswith('^'):
        ticker = symbol
    else:
        ticker = f"{symbol}.NS"

    print(f"\nFetching data for {ticker}...")
    df = yf.download(ticker, period=period, interval="1d", progress=False)

    if df.empty:
        print(f"No data found for {ticker}. Check the symbol.")
        return None

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df.dropna(inplace=True)
    return df

def identify_patterns(df):
    candle_names = talib.get_function_groups()['Pattern Recognition']
    for candle in candle_names:
        pattern_func = getattr(talib, candle)
        df[candle] = pattern_func(df['Open'], df['High'], df['Low'], df['Close'])
    return df, candle_names

def analyze_5_day_window(df, candle_names, end_date_mode):
    """
    Analyzes a 5-day window based on the mode.
    """
    if end_date_mode == '1': # TODAY
        window_df = df.tail(5)
        title_suffix = "(Ending TODAY)"
    elif end_date_mode == '2': # YESTERDAY
        window_df = df.iloc[-6:-1]
        title_suffix = "(Ending YESTERDAY)"
    else:
        print("Invalid selection.")
        return

    print(f"\n{'='*60}")
    print(f"   5-DAY PATTERN ANALYSIS {title_suffix}")
    print(f"{'='*60}")

    for date, row in window_df.iterrows():
        date_str = date.strftime('%Y-%m-%d')
        close_price = row['Close']

        active_patterns = []
        for candle in candle_names:
            value = row[candle]
            if value != 0:
                clean_name = candle.replace('CDL', '')
                sentiment = "🟢 BULL" if value > 0 else "🔴 BEAR"
                desc = pattern_desc.get(candle, clean_name)
                active_patterns.append(f"{clean_name} [{sentiment}]")

        print(f"Date: {date_str} | Close: {close_price:.2f}")
        if active_patterns:
            for p in active_patterns:
                print(f"  -> Found: {p}")
        else:
            print("  -> No significant patterns.")
        print("-" * 40)

# --- Main Execution (Fixed Loop) ---
if __name__ == "__main__":
    stock_symbol = input("Enter Symbol (e.g. RELIANCE, ^NSEI): ")

    df = get_nse_data(stock_symbol)

    if df is not None:
        df_patterns, pattern_list = identify_patterns(df)

        # Ask only once
        print("\nSelect Analysis Base Date:")
        print("1: Today (Analyze last 5 days ending Today)")
        print("2: Yesterday (Analyze last 5 days ending Yesterday)")

        choice = input("Enter Choice (1 or 2): ")

        if choice == '1' or choice == '2':
            analyze_5_day_window(df_patterns, pattern_list, end_date_mode=choice)
        else:
            print("Invalid input. Please restart the script and enter 1 or 2.")

        # input("\nPress Enter to exit...")

ModuleNotFoundError: No module named 'talib'